<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">Used Car Sales Price Prediction Competition

##### <p align="center">*Note: Dataset adapted from N. Birla's public dataset ['Vehicle Dataset'](https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho)* 
---
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data  with Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [1]:
#install aimodelshare library
! pip install aimodelshare --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 377 kB 7.5 MB/s 
     |████████████████████████████████| 7.7 MB 14.7 MB/s 
     |████████████████████████████████| 12.8 MB 26.4 MB/s 
     |████████████████████████████████| 131 kB 44.0 MB/s 
     |████████████████████████████████| 440 kB 20.1 MB/s 
     |████████████████████████████████| 5.2 MB 11.4 MB/s 
     |████████████████████████████████| 302 kB 48.1 MB/s 
     |████████████████████████████████| 178 kB 27.1 MB/s 
     |████████████████████████████████| 146 kB 41.7 MB/s 
     |████████████████████████████████| 96 kB 3.9 MB/s 
     |████████████████████████████████| 127 kB 53.9 MB/s 
     |████████████████████████████████| 278 kB 43.8 MB/s 
     |████████████████████████████████| 22.3 MB 14.5 MB/s 
     |████████████████████████████████| 78 kB 3.3 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |██████████████████████████████

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/used_car_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Separate data into X_train, y_train, and X_test
import pandas as pd

X_train = pd.read_csv("/content/used_car_competition_data/training_data.csv")
y_train = X_train['selling_price']
X_train.drop(['selling_price'], axis=1, inplace=True)

X_test=pd.read_csv("/content/used_car_competition_data/test_data.csv")

X_train.head()

,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,2011,84000,Diesel,Individual,Manual,Second Owner,20.00,1399.0,68.00,5.0
1,2014,74537,Petrol,Individual,Manual,Second Owner,20.51,998.0,67.04,5.0
2,2008,110000,Petrol,Individual,Manual,Second Owner,17.70,1497.0,78.00,5.0
3,2019,5000,Petrol,Individual,Automatic,First Owner,21.70,998.0,67.00,5.0
4,2017,45000,Diesel,Dealer,Automatic,First Owner,19.33,1999.0,177.00,5.0


##2.   Preprocess data using Sklearn Column Transformer / Write and Save Preprocessor function


In [4]:
# In this case we use Sklearn's Column transformer in our preprocessor function

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Preprocess data using sklearn's Column Transformer approach

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['km_driven', 'mileage', 'engine', 'max_power', 'year']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #'imputer' names the step
    ('scaler', StandardScaler())])

categorical_features = ['fuel', 'seller_type', 'transmission', 'owner', 'seats']

# Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [5]:
# Write function to transform data with preprocessor 
# In this case we use sklearn's Column transformer in our preprocessor function

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [6]:
# check shape of X data 
preprocessor(X_train).shape

(6502, 27)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor(n_estimators = 500, max_depth = 3, random_state=0)
model.fit(preprocessor(X_train), y_train) # Fitting to the training set.
model_predictions = model.predict(preprocessor(X_train))
r2_score(model_predictions, y_train)

0.8542251611959975

#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of features in preprocessed data

onnx_model = model_to_onnx(model, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)


with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://tv6fawq47b.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this Used Car Sales Prediction Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted car prices) (Model 1)
predicted_values = model.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_values)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 3

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,username,version
0,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,AIModelShare,1
1,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,AIModelShare,3
2,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,AIModelShare,2


## 5. Repeat submission process to improve place on leaderboard


In [14]:
# Create model 2 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model_2 = RandomForestRegressor(n_estimators = 1000, max_depth = 2, random_state=0)
model_2.fit(preprocessor(X_train), y_train) # Fitting to the training set.
model_predictions = model_2.predict(preprocessor(X_train))
r2_score(model_predictions, y_train) 

0.715128519145602

In [15]:
# Save Model 2 to .onnx file

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of features in preprocessed data

onnx_model = model_to_onnx(model_2, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString()) 

In [16]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 4

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670


In [17]:
# Compare two or more models
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)

,param_name,default_value,model_version_1,model_version_2
0,bootstrap,True,True,True
1,ccp_alpha,0.000000,0.000000,0.000000
2,criterion,mse,mse,mse
3,max_depth,None,3,2
4,max_features,auto,auto,auto
5,max_leaf_nodes,None,None,None
6,max_samples,None,None,None
7,min_impurity_decrease,0.000000,0.000000,0.000000
8,min_impurity_split,None,None,None
9,min_samples_leaf,1,1,1


In [18]:
# Submit a third model using GridSearchCV

from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'n_estimators': np.arange(100, 300, 500),'max_depth':[1, 3, 5]} #np.arange creates sequence of numbers for each k value

gridmodel = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, cv=10)

#use meta model methods to fit score and predict model:
gridmodel.fit(preprocessor(X_train), y_train)

#extract best score and parameter by calling objects "best_score_" and "best_params_"
print("best mean cross-validation score: {:.3f}".format(gridmodel.best_score_))
print("best parameters: {}".format(gridmodel.best_params_))


best mean cross-validation score: 0.933
best parameters: {'max_depth': 5, 'n_estimators': 100}


In [19]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(gridmodel, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

with open("gridmodel.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [20]:
#Submit Model 3: 

#-- Generate predicted values
prediction_labels = gridmodel.predict(preprocessor(X_test))

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "gridmodel.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 5

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670


In [21]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,username,version
0,7671996.51,2769.84,1677.97,0.94,sklearn,False,False,RandomForestRegressor,AIModelShare,5
1,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,AIModelShare,1
2,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,AIModelShare,3
3,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,AIModelShare,2
4,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,AIModelShare,4


In [23]:
# Compare two or more models
data=mycompetition.compare_models([4, 1, 3], verbose=1)
mycompetition.stylize_compare(data)

,param_name,default_value,model_version_4,model_version_1,model_version_3
0,bootstrap,True,True,True,True
1,ccp_alpha,0.000000,0.000000,0.000000,0.000000
2,criterion,mse,mse,mse,mse
3,max_depth,None,2,3,3
4,max_features,auto,auto,auto,auto
5,max_leaf_nodes,None,None,None,None
6,max_samples,None,None,None,None
7,min_impurity_decrease,0.000000,0.000000,0.000000,0.000000
8,min_impurity_split,None,None,None,None
9,min_samples_leaf,1,1,1,1


In [24]:
# Here are several classic ML architectures you can choose from to experiment with next:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor


#Example code to fit model:
model = GradientBoostingRegressor(n_estimators=50, learning_rate=1.0,
    max_depth=1, random_state=0).fit(preprocessor(X_train), y_train)
model.score(preprocessor(X_train), y_train)

# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(model, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

#-- Generate predicted values (a list of predicted labels "real" or "fake")
prediction_labels = model.predict(preprocessor(X_test))

# Submit model to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 6

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670
